# Tutorial on the ExcludeMaskRegion objects

In this tutorial, we go over some details on the usage of the ExcludeMaskRegion objects that can be used to clean mask from specified spectral regions. 

## 0. Importing Necessary Packages

In [ ]:
import specpolFlow as pol
import pandas as pd
import matplotlib.pyplot as plt

## 1. Structure of an `ExcludeMaskRegion` Object

TODO: Explain the structure of the object, and the structure of the corresponding file

## 2. Creating Your Own 

TODO

## 3. Use the Default Functions

TODO

In [ ]:
reg1 = pol.get_Balmer_regions_default(velrange=100)
pd.DataFrame(reg1.to_dict())

In [ ]:
reg2 = pol.get_telluric_regions_default()

pd.DataFrame(reg2.to_dict())

## 4. Combining Multiple `ExcludeMaskRegions`

TODO

In [ ]:
concat = reg1 + reg2

pd.DataFrame(concat.to_dict())

## 5. Save to Files & Read from Files

TODO

In [ ]:
concat.save('ExcludeMaskRegionsClass_tutorialfiles/ExcludeMaskRegions.dat')

In [ ]:
reg = pol.read_exclude_mask_regions('ExcludeMaskRegions_tutorialfiles/ExcludeMaskRegions.dat')
pd.DataFrame(reg.to_dict())

## 6. Using `ExcludeMaskRegions` Object to Clean a Mask

TODO: write some text here

In [ ]:
mask = pol.read_mask('ExcludeMaskRegions_tutorialfiles/T27000G35_depth0.02.mask')

fig, ax = plt.subplots(1,1)
ax.scatter(mask.wl, 1-mask.depth, s=10, label='Full mask')
ax.set_xlabel('Wavelength (nm)')

mask_clean = mask.clean(reg).prune()
ax.scatter(mask_clean.wl, 1-mask_clean.depth, s=10, label='Cleaned Mask')

ax.legend(loc=0)

In [ ]:
print(mask_clean.iuse)